# Scraping

El *scraping* es el arte de descargarse el contenido de una web a partir del HTML de la página. Como todos sabemos es mucho más fácil acceder a los datos de un sitio si dispone de una API, nos permite enganchar nuestro programa directamente y podemos obtener información actualizada en tiempo real; pero muchas veces la realidad no es tan sencilla y sólo disponemos de los datos a través de la interfaz que vemos en el navegador.

Debajo de ese conjunto de textos con colores, tipografías, imagénes y trucos para embelesar al usuario, lo que se esconden es un lenguaje de marcado que se llama HTML (*Hypertext markup language*) que tiene un aspecto parecido a este:

```html
<html>
    <head>
        <!-- Comentario: Este título se mostrará en la barra superior del navegador -->
        <title>Wikipedia | HTML</title>
    </head>
    <body>
        <!-- Todo lo incluido dentro de la etiqueta 'body' es lo vemos en el navegador -->
        <div id="content">
            <h1>Ejemplo de HTML</h1>
            <div class="section">
                <h2>Primera sección</h2>
                <p>Y esto es un párrafo con algo en <strong>negrita</strong> y poco más.</p>
            </div>
            <div>
                <h2>Segunda sección</h2>
                <p>En este otro párrafo pongo un <a href="https://lingwars.github.io/blog">link a Lingwars</a>.</p>
            </div>
        </div>
    </body>
</html>
```

A través de expresiones regulares y cadenas XPath podemos acceder directamente a los contenidos que nos interesan dentro de esta estructura jerárquica. Los ordenadores son muy buenos haciendo estas tareas, sólo tenemos que decirles dónde buscar.

**Nota.-** Este cuaderno es una extensión del artículo publicado en el blog de Lingẅars [Scrapeando la web con Python](http://lingwars.github.io/blog/scrape-xpath.html), puedes acudir allí para tener más detalles, aquí nos centraremos más en el código.

## 1. Descargarse el contenido de la web

El primer paso, en cualquier proceso de scraping es descargarse el contenido de la web que nos interesa, por ejemplo:
```
http://www.elmundo.es/internacional.html
```

In [1]:
import requests

url = "http://www.elmundo.es/internacional.html"

def download(url):
    r = requests.get(url)
    if r.status_code != 200:
        print("! Error {} retrieving url {}".format(r.status_code, url))
    return r

page = download(url)

print("A continuación una muestra del HTML que nos hemos descargado (sólo los primeros 150 caracteres):")
print(page.text[:150])

A continuación una muestra del HTML que nos hemos descargado (sólo los primeros 150 caracteres):
<!DOCTYPE html>
<html lang="es">
<head>
<meta http-equiv="Content-Type" content="text/html; charset=iso-8859-15"/>
<meta http-equiv="X-UA-Compatible" 


Si abres la URL anterior en tu navegador [http://www.elmundo.es/internacional.html](http://www.elmundo.es/internacional.html) y le das a "ver código fuente" (click con el botón derecho del ratón > ver código fuente/inspeccionar elemento) podrás comprobar que el HTML anterior se corresponde con el de la web de El País. Perfecto, ya tenemos el contenido sobre el que vamos a trabajar.

## 2. Accediendo a los datos que nos interesan: XPath

Lo siguiente que tenemos que hacer es recuperar los datos que nos interesan. Para ello es extremadamente útil conocer el funcionamiento de XPath y sus posibilidades. ¿Vinistéis al taller que hicimos de scraping? ¡No? Bueno... menos mal que el XPath no nos lo hemos inventado nosotros sino que podéis encontrar muchísima información online.

Nosotros lo que vamos a hacer ahora es recuperar todos los enlaces que hay en el HTML anterior (la portada de internacional de El Mundo), sin duda, entre estos enlaces estarán las noticias de actualidad más importantes, ¡es la portada!

Si te fijas en el HTML podrás comprobar que los enlaces tienen una estructura como la siguiente:

```html
<a href="https://lingwars.github.io/blog">link a Lingwars</a>
```

Es decir, siempre aparecen en un nodo `a` con un atributo `href` que es la dirección a la que apuntan y un texto dentro del nodo que es el que se muestra al usuario (convencionalmente, se muestra subrayado y en azul).

Para recuperar todos los enlaces podemos, por lo tanto, utilizar una cadena xpath como la siguiente:

```xpath
//a/@href
```

Vamos a probarlo de verdad:

In [2]:
from lxml import html

xpath_string = '//a/@href'

# Necesito cargar mi html en una estructura de árbol XML
tree = html.fromstring(page.content)
# Hago la llamada al xpath para obtener los resultados
results = tree.xpath(xpath_string)

print("Hay {} enlaces en esta web!".format(len(results)))
for u in results[:20]:  # Voy a mostrar sólo los 20 primeros
    print(u)

ImportError: No module named 'lxml'

## 3. Filtrar, eliminar duplicados,...

### regex
En la muestra anterior podemos ver que hay algunos enlaces que no se corresponden con noticias, o que abandonan la sección de internacional que es la que nos interesaba,... tenemos que intentar reconocer cuál es el patrón que comparten todos los enlaces que nos interesan.

Una vez que lo hayamos identificado, podemos construir una expresión **regex** (o varias) que los identifiquen. Sugerencia de la casa:

```
article_internacional_patterns = [
    re.compile('https?:\/\/(www.)?elmundo.es\/internacional\/(?P<year>\d{4})\/(?P<month>\d{2})\/(?P<day>\d{2})\/(?P<uuid>[\d\w]+).html'),
    ]
```

Y con este patrón podemos filtrar todas las URLs anteriores para quedarnos sólo con las que sean *internacional*.

In [33]:
import re

pattern = re.compile('https?:\/\/(www.)?elmundo.es\/internacional\/(?P<year>\d{4})\/(?P<month>\d{2})\/(?P<day>\d{2})\/(?P<uuid>[\d\w]+).html')

results = filter(lambda u: pattern.match(u), results)
results = list(results)

print("He encontrado {} URL que encajen en el patrón anterior (muestro sólo 20):".format(len(results)))
for u in results[:20]:
    print(u)

He encontrado 145 URL que encajen en el patrón anterior (muestro sólo 20):
http://www.elmundo.es/internacional/2016/07/15/578940e3268e3e1b0b8b45fb.html
http://www.elmundo.es/internacional/2016/07/15/578940e3268e3e1b0b8b45fb.html
http://www.elmundo.es/internacional/2016/07/15/578940e3268e3e1b0b8b45fb.html#comentarios
http://www.elmundo.es/internacional/2016/07/16/57898a68e5fdea87598b468d.html
http://www.elmundo.es/internacional/2016/07/16/5789fba1468aebba388b4620.html
http://www.elmundo.es/internacional/2016/07/16/5789fba1468aebba388b4620.html
http://www.elmundo.es/internacional/2016/07/16/5789fba1468aebba388b4620.html
http://www.elmundo.es/internacional/2016/07/16/5789e5b546163f54368b464b.html
http://www.elmundo.es/internacional/2016/07/16/5789e5b546163f54368b464b.html
http://www.elmundo.es/internacional/2016/07/16/5789e5b546163f54368b464b.html#comentarios
http://www.elmundo.es/internacional/2016/07/16/578932ba268e3ed26b8b46aa.html
http://www.elmundo.es/internacional/2016/07/16/578932b

### Eliminar duplicados
Pero puede haber duplicados, vamos a librarnos de ellos con el siguiente código:

In [34]:
articles = set()  # Los contenedores de tipo set no permiten elementos repetidos ;D
for r in results:
    r = r.rsplit("#", 1)[0]
    r = r.rsplit("?", 1)[0]
    articles.add(r)

¿Cuántos articulos crees que nos han quedado? Vamos a verlo

In [35]:
print("He encontrado {} URL que encajen en el patrón anterior:".format(len(articles)))
for u in articles:
    print(u)

He encontrado 57 URL que encajen en el patrón anterior:
http://www.elmundo.es/internacional/2016/07/13/5786a3a8e2704ebe158b4581.html
http://www.elmundo.es/internacional/2016/07/13/57869f66468aebb6358b45fd.html
http://www.elmundo.es/internacional/2016/07/16/57898a68e5fdea87598b468d.html
http://www.elmundo.es/internacional/2016/07/15/57892f1146163f6f7b8b459c.html
http://www.elmundo.es/internacional/2016/07/16/578994dbe5fdea1a788b45fd.html
http://www.elmundo.es/internacional/2016/07/15/57889ec6e2704e59178b4597.html
http://www.elmundo.es/internacional/2016/07/16/578932ba268e3ed26b8b46aa.html
http://www.elmundo.es/internacional/2016/07/13/57862ab146163fe97a8b4645.html
http://www.elmundo.es/internacional/2016/07/13/578649f422601d2e4c8b45a0.html
http://www.elmundo.es/internacional/2016/07/15/57889f8746163f54368b45b3.html
http://www.elmundo.es/internacional/2016/07/15/5788a7e4e2704e6b588b45ac.html
http://www.elmundo.es/internacional/2016/07/15/5787e64b46163f97198b4646.html
http://www.elmundo.e

Hemos conseguido los enlaces a todas las noticias que hay en la portada de la sección internacional de El Mundo.

## 4. Navegar por los enlaces
Pero somos ambiciosos y realmente lo que queríamos era obtener los titulares y el texto de cada uno de los artículos. ¿Puedo hacerlo con Python? ¡Cómo no, pues claro!

Fíjate que ahora tenemos una lista de URLs que apuntan a todas las noticias que estaban en portada de la sección internacional y, por si te has olvidado, te recuerdo que este cuaderno empezó con una URL... Básicamente podemos realizar el proceso que hemos seguido desde el principio para cada una de estas URLs y, utilizando los XPath adecuados, obtendremos toda la información de estas noticias.

Vamos a definir unas cuantas variables útiles y luego iremos a por el código. Si analizamos el HTML de cualquier noticia de las anteriores, podemos construir algunos XPath que nos permitirán recuperar alguna información relevante:

In [38]:
xpath_string = {'title': "//article/h1[@itemprop='headline']/text()",
                'summary': "//article/div[@itemprop='articleBody']/p[@class='summary-lead']//text()",
                'author': "//footer/ul/li[@itemprop='name']//text()",
                'location': "//footer/ul/li[@itemprop='address']//text()",
                'datetime': "//article/div[@itemprop='articleBody']/time//text()",
                'content': "//article/div[@itemprop='articleBody']/p[not(@class='summary-lead')]//text()",
                 }

Pues hagamos un bucle que recorra todos los enlaces que teníamos guardados en la variable `articles` y vaya ejecutando estos xpaths para obtener la información

In [42]:
for i, article in enumerate(articles):
    print("Article #{}".format(i))
    print(" - url: {}".format(article))
    data = download(article)
    tree = html.fromstring(data.content)
    
    for key, value in xpath_string.items():
        item = tree.xpath(value)
        if not isinstance(item, str):
            if key == 'summary':
                item = '. '.join(item).strip()
            else:
                item = ''.join(item).strip()
        print(" - {}: {}".format(key, item[:60]))

Article #0
 - url: http://www.elmundo.es/internacional/2016/07/13/5786a3a8e2704ebe158b4581.html
 - datetime: 13/07/2016 22:51
 - content: "La Organización de Estados Americanos (OEA) nos ha decepcio
 - location: 
 - title: "La OEA nos ha decepcionado al no ser capaz de construir uni
 - summary: Queremos convertir Bolivia en el centro energético de Améric
 - author: ROCÍO GALVÁN
Article #1
 - url: http://www.elmundo.es/internacional/2016/07/13/57869f66468aebb6358b45fd.html
 - datetime: 13/07/2016 22:22
 - content: Theresa May entró este miércoles en el número 10 de Downing 
 - location: Londres
 - title: Theresa May reta a Europa
 - summary: La primera ministra británica coloca al eurófobo Boris Johns
 - author: CARLOS FRESNEDA
Article #2
 - url: http://www.elmundo.es/internacional/2016/07/16/57898a68e5fdea87598b468d.html
 - datetime: 16/07/2016 03:25
 - content: El presidente del Gobierno, Mariano Rajoy, ha trasladado a T
 - location: 
 - title: Rajoy: "España apoya el orden constituci

# Conclusión
Espero que este cuaderno te haya sido útil y que te haya ayudado el hecho de poder interactuar con el código al tiempo que lees el texto de las explicaciones. Y también que hayas sido consciente del poder de las expresiones regulares (regex) junto con el xpath para las labores de scraping.
